In [47]:
import os 
import pandas as pd
import sys


In [48]:
# -*- coding: utf-8 -*-
import redis

if __name__ == '__main__':
    import sys
    sys.path.append('../')

class RedisWrapper(object):
    """
    Redis连接类
    """
    def __init__(self,db=0):
        self.conn = None
        self._set_db_conn(db)

    def _set_db_conn(self, db=0):
        connection_pool = redis.ConnectionPool(host='172.18.18.165', port='19000', db=1)
        self.conn = redis.StrictRedis(connection_pool=connection_pool)

    def get_conn(self):
        db = self.conn
        return db

redis_instance = RedisWrapper().get_conn()




In [49]:
print(redis_instance.hmset('test', {'1233':"123****", '12':'12&&&&&&&'}))
#import pdb; pdb.set_trace()
print(redis_instance.hget('test', '1233'))
print(redis_instance.hget('test', '12'))
print(redis_instance.get('1233'))

True
b'123****'
b'12&&&&&&&'
None


In [25]:
from proxy_config import PROXIES_UNICOM_IP_POOLS, PROXIES_IP_POOLS
from crawler_config import CHANNEL_MAPPING


class Redis_que(object):
    """
    Redis参数入队初始化
    """
    def __init__(self):
        self.conn = RedisWrapper(1).get_conn()

    def init_redis(self):
        self._set_proxies_que()
        self._set_channel_que()

    def _set_proxies_que(self):
        p = self.conn.pipeline(transaction=False)
        p.delete('bmp_crawl_proxies_cmcc')
        p.delete('bmp_crawl_proxies_cucc')
        pattern = lambda x,y: [p.lpush(y, i) for i in x]
        pattern(PROXIES_IP_POOLS, 'bmp_crawl_proxies_cmcc')
        pattern(PROXIES_UNICOM_IP_POOLS, 'bmp_crawl_proxies_cucc')
        p.execute()   

    def _set_channel_que(self):
        self.conn.hmset('bmp_crawl_channel', CHANNEL_MAPPING)

    def pop_proxies(self,ip_type='bmp_crawl_proxies_cmcc'):
        ip = self.conn.rpoplpush(ip_type, ip_type)
        # print 'proxies',ip,self.conn.llen(ip_type)
        return ip

    def get_channel(self):
        return self.conn.hgetall('bmp_crawl_channel')

    def set_data(self, key, data):
        self.conn.hmset(key, data)

    def get_data(self, key):
        self.conn.hgetall(key)

    def data_exists(self, key, field):
        return self.conn.hexists(key, field)
    
redis_que = Redis_que()
redis_que.init_redis()


In [30]:

print(redis_que.pop_proxies())

b'squidsz01.dianhua.cn'


In [31]:
class RedisQueue(object):
    def __init__(self, name, namespace='queue', **redis_kwargs):
       # redis的默认参数为：host='localhost', port=6379, db=0， 其中db为定义redis database的数量
       self.db= redis_instance
       self.key = '%s:%s' %(namespace, name)

    def qsize(self):
        return self.db.llen(self.key)  # 返回队列里面list内元素的数量

    def put(self, item):
        self.db.rpush(self.key, item)  # 添加新元素到队列最右方

    def get_wait(self, timeout=None):
        # 返回队列第一个元素，如果为空则等待至有元素被加入队列（超时时间阈值为timeout，如果为None则一直等待）
        item = self.db.blpop(self.key, timeout=timeout)
        # if item:
        #     item = item[1]  # 返回值为一个tuple
        return item

    def get_nowait(self):
        # 直接返回队列第一个元素，如果队列为空返回的是None
        item = self.db.lpop(self.key)  
        return item
redis_queue_tel_for_name = RedisQueue('tel_for_name',db=6)  # 新建队列名为rq
redis_queue_tel_for_name_ret = RedisQueue('tel_for_name_ret',db=6) 
redis_queue_name_for_tel = RedisQueue('name_for_tel',db=6) 
redis_queue_name_for_tel_ret = RedisQueue('name_for_tel_ret',db=6) 

In [34]:
print(redis_queue_tel_for_name.qsize())
print(redis_queue_tel_for_name.put('hahahahahaha'))
print(redis_queue_tel_for_name.qsize())
print(redis_queue_tel_for_name.get_nowait())
print(redis_queue_tel_for_name.qsize())

0
None
1
b'hahahahahaha'
0


In [38]:
redis_instance.hset('myhash ','myhash ',5)

1

In [42]:
redis_instance.hincrby('myhash ','myhash ',-10)

0